# RoleHierarchy

> a class based approach to handling role hierarchy

Assumes role descriptions end with `{description} - h{hiearchy_number}` where a higher number ranks higher in the hierarchy.


In [ ]:
# | default_exp integrations.RoleHierarchy

In [ ]:
# | exporti
import domolibrary.classes.DomoRole as dmr

In [ ]:
# |export
def extract_role_hierarchy(
    role: dmr.DomoRole, hierarchy_delimiter, debug_prn: bool = False
) -> dmr.DomoRole:  # augments the domo role with a hierarchy INT attribute

    description_arr = role.description.split(hierarchy_delimiter)

    if len(description_arr) != 1:
        hierarchy = int(description_arr[1])

    elif role.is_system_role:
        hierarchy = (5 - role.id) * 2 + 1

    else:
        hierarchy = 0

    role.hierarchy = hierarchy

    if debug_prn:
        print(
            {
                "description_arr": description_arr,
                "role_id": role.id,
                "hierarchy": hierarchy,
            }
        )

    return role

In [ ]:
# | export
async def get_roles_w_hierarchy(
    auth,
    hierarchy_delimiter=" - h",  # post fix to delimit hierarchy number.  assumes scale of 1:10, system accounts will be included.
    debug_prn: bool = False,
    debug_api: bool = False,
):
    """gets instance roles and adds an attribute hierarchy"""

    domo_roles = await dmr.DomoRoles.get_roles(auth=auth, debug_api=debug_api)
    return [
        extract_role_hierarchy(
            role=role, hierarchy_delimiter=hierarchy_delimiter, debug_prn=debug_prn
        )
        for role in domo_roles
    ]

In [ ]:
import os
import pandas as pd
import domolibrary.client.DomoAuth as dmda


token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


domo_roles = await get_roles_w_hierarchy(auth=token_auth)


pd.DataFrame(
    [
        {
            "role_name": role.name,
            "role_id": role.id,
            "description": role.description,
            "hierarchy": role.hierarchy,
        }
        for role in sorted(domo_roles, key=lambda x: x.hierarchy, reverse=True)
    ]
)

,role_name,role_id,description,hierarchy
0,Admin,1,Full access to everything,9
1,manual_super_admin,275763436,Full access to everything - h9,9
2,Privileged,2,Full access except for editing users and setti...,7
3,Editor,3,"Can edit Cards, Pages, DataSets, and Dataflows",5
4,Participant,4,Read only access to Cards and Pages,3
5,Social,5,Access limited to social features,1
6,super_admin_v3,104295428,upsert via DomoLibrary,0
7,super_admin,810756122,all grants - updated on 2023-08-17,0
8,dl_department_admin,1563101750,deployed via domo_library script - updated 202...,0
9,dl_test,1662852841,deployed via domo_library script - updated 202...,0


In [ ]:
# | export
async def calc_role(
    current_role_id,
    new_role_name,
    auth,
    hierarchy_delimiter=" - h",
    is_alter_system_roles: bool = False,  # by default calc role will not apply to system roles and will always update to a system role
    debug_prn: bool = False,
):
    """compares current role to new role hierarchy and returns the higher one.  will not adjust system roles"""

    instance_roles = await get_roles_w_hierarchy(
        auth=auth, hierarchy_delimiter=hierarchy_delimiter
    )

    current_role = next((role for role in instance_roles if role.id == current_role_id))

    if current_role.is_system_role and not is_alter_system_roles:
        print(f"{current_role.name} is a system role -- no changes")
        return current_role

    expected_role = next(
        (role for role in instance_roles if role.name == new_role_name), None
    )

    if not expected_role:
        raise Exception(f"{new_role_name} not found in {auth.domo_instance}")

    if current_role.hierarchy >= expected_role.hierarchy:
        if debug_prn:
            print(
                f"do nothing:  {current_role.name} - {current_role.hierarchy} exceeds or equals {expected_role.name} - {expected_role.hierarchy}"
            )
        return current_role

    if debug_prn:
        print(
            f"upgrade role: {current_role.name} - {current_role.hierarchy} to a {expected_role.name} - {expected_role.hierarchy}"
        )
    return expected_role

In [ ]:
import os
import domolibrary.client.DomoAuth as dmda


token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

print(await calc_role(3, "manual_super_admin", auth=token_auth, debug_prn=True))
print("\n")
print(
    await calc_role(
        3,
        "manual_super_admin",
        is_alter_system_roles=True,
        auth=token_auth,
        debug_prn=True,
    )
)
print("\n")
try:
    print(
        await calc_role(
            3, "social", is_alter_system_roles=True, auth=token_auth, debug_prn=True
        )
    )
except Exception as e:
    print(e)
print("\n")
print(
    await calc_role(
        3, "Social", is_alter_system_roles=True, auth=token_auth, debug_prn=True
    )
)

Editor is a system role -- no changes
DomoRole(id=3, name='Editor', description='Can edit Cards, Pages, DataSets, and Dataflows', is_system_role=True, is_default_role=False, grant_ls=[], membership_ls=[])


upgrade role: Editor - 5 to a manual_super_admin - 9
DomoRole(id=275763436, name='manual_super_admin', description='Full access to everything - h9', is_system_role=0, is_default_role=False, grant_ls=[], membership_ls=[])


social not found in domo-community


do nothing:  Editor - 5 exceeds or equals Social - 1
DomoRole(id=3, name='Editor', description='Can edit Cards, Pages, DataSets, and Dataflows', is_system_role=True, is_default_role=False, grant_ls=[], membership_ls=[])
